# ASSIGNMENT-3

## WEB SCRAPING

In [1]:
!pip install selenium

In [2]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings 
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,InvalidArgumentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests

import time


Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
# execption handling
try:
    driver = webdriver.Chrome(r"chromedriver.exe")
    driver.get("www.amazon.in.")
except InvalidArgumentException as e:
    print('InvalidArgumentException rasied',e)
    driver = webdriver.Chrome(r"chromedriver.exe")
    driver.get("https://www.amazon.in/")    

InvalidArgumentException rasied Message: invalid argument
  (Session info: chrome=103.0.5060.66)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E35FD3+2187219]
	Ordinal0 [0x00DCE6D1+1763025]
	Ordinal0 [0x00CE3D40+802112]
	Ordinal0 [0x00CD6326+746278]
	Ordinal0 [0x00CD4E3A+740922]
	Ordinal0 [0x00CD50FD+741629]
	Ordinal0 [0x00CE5644+808516]
	Ordinal0 [0x00D3D66D+1169005]
	Ordinal0 [0x00D2CB8C+1100684]
	Ordinal0 [0x00D3CFC2+1167298]
	Ordinal0 [0x00D2C9A6+1100198]
	Ordinal0 [0x00D06F80+946048]
	Ordinal0 [0x00D07E76+949878]
	GetHandleVerifier [0x010D90C2+2721218]
	GetHandleVerifier [0x010CAAF0+2662384]
	GetHandleVerifier [0x00EC137A+526458]
	GetHandleVerifier [0x00EC0416+522518]
	Ordinal0 [0x00DD4EAB+1789611]
	Ordinal0 [0x00DD97A8+1808296]
	Ordinal0 [0x00DD9895+1808533]
	Ordinal0 [0x00DE26C1+1844929]
	BaseThreadInitThunk [0x74766359+25]
	RtlGetAppContainerNamedObjectPath [0x76FC7C24+228]
	RtlGetAppContainerNamedObjectPath [0x76FC7BF4+180]



In [4]:
search_product = input("Search Product : ")
search = driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
search.send_keys(search_product)
time.sleep(5)
clk = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
clk.click()
    

Search Product : watches


Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
# creating empty list for product details
p_brand = []
p_name = []
p_price = []
p_exchange = []
p_delivery = []
p_availability = []
p_url = []

In [6]:
# scraping urls

start=0
end=3
for page in range(start,end):#for loop for scrapping 3 page
    url=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')       #scraping urls
    for i in url:
        p_url.append(i.get_attribute("href"))                        #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')     #scraping the list of buttons from the page
    nxt_button.click()
    time.sleep(2)

In [7]:
first_p = driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[64]/div/div/span/a[2]')
first_p.click()

In [9]:
len(p_url)

188

In [20]:
# scraping product name
for url in p_url:
    driver.get(url)
    time.sleep(2)
# Extracting Brand from xpath
    try:
        brand = driver.find_element(By.XPATH,'//a[@id="bylineInfo"]')      
        p_brand.append(brand.text.replace('Visit the ','').replace(' Store','').replace('Brand: ',''))
    except NoSuchElementException:
        p_brand.append('-')    
# extracting name from xpath
    try:
        name = driver.find_element(By.XPATH,'//span[@id="productTitle"]')
        p_name.append(name.text)
    except NoSuchElementException:
        p_name.append('-')
# extracting price from xpath
    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-price a-text-price a-size-medium apexPriceToPay"]')
        p_price.append(price.text)
    except NoSuchElementException:
        try:
            price1 = driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center"]')
            p_price.append(price1.text.replace('/n00',' '))
        except NoSuchElementException:
            p_price.append('-')  
# extracting return days
    try:
        exchange = driver.find_element(By.XPATH,'//div[@class="a-column a-span3 a-text-center icon-container icon-grid-width celwidget"]')
        p_exchange.append(exchange.text)
    except NoSuchElementException:
        p_exchange.append('No Exchange/Return')        
# extracting delivery time
    try:
        delivery = driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
        p_delivery.append(delivery.text)
    except NoSuchElementException:
        p_delivery.append('-')       
# extracting availablity
    try:
        available = driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
        p_availability.append(available.text)
    except NoSuchElementException:
        p_availability.append('-')        

In [49]:
newlist = []
for f in p_price:
    f = f.replace('\n00','')
    newlist.append(f)
p_price = newlist

In [66]:
# creating dataframe for our data
df = pd.DataFrame({'BRAND':p_brand,
                   'NAME':p_name,
                   'PRICE':p_price,
                   'AVAILABILITY':p_availability,
                   'DELIVERY':p_delivery,
                   'EXCHANGE/RETURN':p_exchange,
                   'URL':p_url
                  },index=list(range(1,189)))
print("PRODUCT DETAILS OF FIRST 3 PAGES ON AMAZON")
df

PRODUCT DETAILS OF FIRST 3 PAGES ON AMAZON


,BRAND,NAME,PRICE,AVAILABILITY,DELIVERY,EXCHANGE/RETURN,URL
1,Noise,Noise Pulse Go Buzz Advanced Bluetooth Calling...,"₹2,499",In stock.,"Fri, 23 Sep",7 Days Replacement,https://www.amazon.in/gp/slredirect/picassoRed...
2,Daniel Wellington,Daniel Wellington Iconic Link Emerald 40mm Ana...,"₹23,999",In stock.,,10 Days Returns,https://www.amazon.in/gp/slredirect/picassoRed...
3,Daniel Wellington,Daniel Wellington Analogue Men's Watch (White ...,"₹19,799",Only 4 left in stock.,,10 Days Returns,https://www.amazon.in/gp/slredirect/picassoRed...
4,Daniel Wellington,Daniel Wellington Classic Analog Black Dial Me...,"₹17,500",Only 2 left in stock.,,10 Days Returns,https://www.amazon.in/gp/slredirect/picassoRed...
5,boAt,"boAt Wave Lite Smartwatch with 1.69"" HD Displa...","₹1,999",In stock.,"Fri, 23 Sep",Free Delivery,https://www.amazon.in/boAt-Wave-Lite-Smartwatc...
...,...,...,...,...,...,...,...
184,Chumbak,Teal by Chumbak Carnival Elephant Wrist Watch,"₹1,247",In stock.,,10 Days Returns,https://www.amazon.in/Chumbak-Aztec-Interchang...
185,TIMEWEAR,TIMEWEAR Analog Number Dial Leather Strap Watc...,₹279,In stock.,,10 Days Returns,https://www.amazon.in/gp/slredirect/picassoRed...
186,DANIEL KLEIN,DANIEL KLEIN Analog Black Dial Men's Watch-DK....,"₹3,172",In stock.,,10 Days Returns,https://www.amazon.in/gp/slredirect/picassoRed...
187,Decode,Decode Analogue Men's Watch (Black Dial Black ...,₹295,In stock.,,Free Delivery,https://www.amazon.in/gp/slredirect/picassoRed...


In [67]:
# saving dataframe as csv file
df.to_csv('watches.csv')

Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [3]:
# connecting to deriver
driver = webdriver.Chrome(r"chromedriver.exe")

In [4]:
driver.get('https://www.google.co.in/')

In [5]:
image = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div/div/div[1]/div/div[2]/a')
image.click()

In [6]:
search = driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')
search.send_keys('fruits')

In [7]:
search_b = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')
search_b.click()

In [9]:
# scrolling window 
driver.execute_script("window.scrollBy(0,1000)","")

In [26]:
# scrapinig urls of images
source = []
img_url = []
img_data = []
image = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')   
for im in image:
    source = im.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_url.append(source)


In [28]:
len(img_url)

53

In [31]:
# downloading fruits images        
for i in range(len(img_url)):
    if i >= 10:
        break
    print('downloading image {0} of {1}'.format(i,10))
    response = requests.get(img_url[i])
    file= open(r"F:\images"+str(i)+".jpg","wb")
    file.write(response.content)

downloading image 0 of 10
downloading image 1 of 10
downloading image 2 of 10
downloading image 3 of 10
downloading image 4 of 10
downloading image 5 of 10
downloading image 6 of 10
downloading image 7 of 10
downloading image 8 of 10
downloading image 9 of 10


In [35]:
# downloading car images
search = driver.find_element(By.XPATH,'//input[@class="og3lId"]')
search.clear()
time.sleep(1)
search.send_keys('cars')       # entering car
time.sleep(2)                     
# click on search
clk = driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
clk.click()
time.sleep(2)
# scrolling window 
driver.execute_script("window.scrollBy(0,1000)","")
time.sleep(2)
# scrapinig url of cars
source2 = []
car_url = []
image = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')   
for im in image:
    source2 = im.get_attribute('src')
    if source2 is not None:
        if(source2[0:4] == 'http'):
            car_url.append(source2)
# downloading car images        
for i in range(len(img_url)):
    if i >= 10:
        break
    print('downloading image {0} of {1}'.format(i,10))
    response = requests.get(car_url[i])
    file= open(r"F:\images"+str(i)+".jpg","wb")
    file.write(response.content)            

downloading image 0 of 10
downloading image 1 of 10
downloading image 2 of 10
downloading image 3 of 10
downloading image 4 of 10
downloading image 5 of 10
downloading image 6 of 10
downloading image 7 of 10
downloading image 8 of 10
downloading image 9 of 10


In [37]:
# downloading ‘Machine Learning’ images
search = driver.find_element(By.XPATH,'//input[@class="og3lId"]')
search.clear()
time.sleep(1)
search.send_keys('Machine Learning')       # entering ‘Machine Learning’
time.sleep(2)                     
# click on search
clk = driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
clk.click()
time.sleep(2)
# scrolling window 
driver.execute_script("window.scrollBy(0,1000)","")
time.sleep(2)
# scrapinig url of ‘Machine Learning’
source3 = []
Machine_Learning_url = []
mimage = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')   
for im in mimage:
    source3 = im.get_attribute('src')
    if source3 is not None:
        if(source3[0:4] == 'http'):
            Machine_Learning_url.append(source3)
# downloading ‘Machine Learning’ images        
for i in range(len(img_url)):
    if i >= 10:
        break
    print('downloading image {0} of {1}'.format(i,10))
    response = requests.get(Machine_Learning_url[i])
    file= open(r"F:\images\New folder"+str(i)+".jpg","wb")
    file.write(response.content) 

downloading image 0 of 10
downloading image 1 of 10
downloading image 2 of 10
downloading image 3 of 10
downloading image 4 of 10
downloading image 5 of 10
downloading image 6 of 10
downloading image 7 of 10
downloading image 8 of 10
downloading image 9 of 10


In [43]:
# downloading Guitar images
search = driver.find_element(By.XPATH,'//input[@class="og3lId"]')
search.clear()
time.sleep(1)
search.send_keys('Guitar')       # entering Guitar
time.sleep(2)                     
# click on search
clk = driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
clk.click()
time.sleep(2)
# scrolling window 
driver.execute_script("window.scrollBy(0,1000)","")
time.sleep(2)
# scrapinig url of Guitar
source4 = []
Guitar_url = []
gimage = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')   
for im in gimage:
    source4 = im.get_attribute('src')
    if source4 is not None:
        if(source4[0:4] == 'http'):
            Guitar_url.append(source4)
# downloading ‘Machine Learning’ images        
for i in range(len(Guitar_url)):
    if i >= 10:
        break
    print('downloading image {0} of {1}'.format(i,10))
    response = requests.get(Guitar_url[i])
    file= open(r"F:\images\New folder\New folder"+str(i)+".jpg","wb")
    file.write(response.content) 

downloading image 0 of 10
downloading image 1 of 10
downloading image 2 of 10
downloading image 3 of 10
downloading image 4 of 10
downloading image 5 of 10
downloading image 6 of 10
downloading image 7 of 10
downloading image 8 of 10
downloading image 9 of 10


In [44]:
# downloading Cakes images
search = driver.find_element(By.XPATH,'//input[@class="og3lId"]')
search.clear()
time.sleep(1)
search.send_keys('Cakes')       # entering Cakes
time.sleep(2)                     
# click on search
clk = driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
clk.click()
time.sleep(2)
# scrolling window 
driver.execute_script("window.scrollBy(0,1000)","")
time.sleep(2)
# scrapinig url of Guitar
source5 = []
Cakes_url = []
cimage = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')   
for im in cimage:
    source5 = im.get_attribute('src')
    if source5 is not None:
        if(source5[0:4] == 'http'):
            Cakes_url.append(source5)
# downloading ‘Machine Learning’ images        
for i in range(len(Cakes_url)):
    if i >= 10:
        break
    print('downloading image {0} of {1}'.format(i,10))
    response = requests.get(Cakes_url[i])
    file= open(r"F:\images\New folder\New folder"+str(i)+".jpg","wb")
    file.write(response.content) 

downloading image 0 of 10
downloading image 1 of 10
downloading image 2 of 10
downloading image 3 of 10
downloading image 4 of 10
downloading image 5 of 10
downloading image 6 of 10
downloading image 7 of 10
downloading image 8 of 10
downloading image 9 of 10


In [45]:
driver.close()

Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [12]:
# connecting to web driver
driver = webdriver.Chrome(r'chromedriver.exe')

In [14]:
# connecting to flipkart
try:
    driver.get('www.flipkart.com')
except InvalidArgumentException:
    driver.get('https://www.flipkart.com/')

In [19]:
# click on cancel button
cancel = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/button')
cancel.click()

In [21]:
# enter smartphone
search_phone = driver.find_element(By.XPATH,'//input[@class="_3704LK"]')
search_phone.send_keys('smartphone')

In [23]:
# click on search
enter = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button')
enter.click()

In [27]:
# scraping urls 
urls = []
phone_urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for i in phone_urls:
    urls.append(i.get_attribute("href"))

In [31]:
# creationg list for storing details of smartphone
Brand_Name = []
Smartphone_name = []
Colour = []
RAM = []
Storage_ROM = []
Primary_Camera = []
Secondary_Camera = []
Display_Size = []
Battery_Capacity = [] 
Price = []

In [128]:
for url in urls:
    driver.get(url)
    time.sleep(2)
    buttoon = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')
    buttoon.click()
# scraping brand of smartphone   
    try:
        brand = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[2]/div[1]/div/div[4]')      
        Brand_Name.append(brand.text.replace('Mobiles',' '))
    except NoSuchElementException:
        Brand_Name.append('-')
# scraping name of phone
    try:
        phone = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[3]/div/div[1]/h1/span')
        Smartphone_name.append(phone.text)
    except NoSuchElementException:
        Smartphone_name.append('-')
# scraping colour
    try:
        colour_phone = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(colour_phone.text)
    except NoSuchElementException:
        Colour.append('-')
# scraping RAM
    try:
        phone_RAM = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(phone_RAM.text)
    except NoSuchElementException:
        RAM.append('-')
#scraping ROM  
    try:
        phone_ROM = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage_ROM.append(phone_ROM.text)
    except NoSuchElementException:
        Storage_ROM.append('-')
# scraping primary camera
    try:
        p_camera = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(p_camera.text)
    except NoSuchElementException:
        Primary_Camera.append('-')  
# scraping sceondary camera
    try:
        s_camera = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li')
        Secondary_Camera.append(s_camera.text)
    except NoSuchElementException:
        Secondary_Camera.append('-') 
# scraping display size
    try:
        d_size = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_Size.append(d_size.text)
    except NoSuchElementException:
        Display_Size.append('-')        
# scraping Battery Capacity
    try:
        b_cap = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr[1]/td[2]/ul/li')
        Battery_Capacity.append(b_cap.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')
# scraping price
    try:
        ph_price = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(ph_price.text)
    except NoSuchElementException:
        Price.append('-')        

In [139]:
df = pd.DataFrame({'BRAND':Brand_Name,
                   'PHONE NAME':Smartphone_name,
                   'COLOUR':Colour,
                   'RAM':RAM,
                   'ROM':Storage_ROM,
                   'PRIMARY CAMERA':Primary_Camera,
                   'SECONDARY CAMERA':Secondary_Camera,
                   'DISPALY SIZE':Display_Size,
                   'BATTERY':Battery_Capacity,
                   'Price':Price,
                   'url':urls},index=list(range(1,len(Price)+1)))
df                 

,BRAND,PHONE NAME,COLOUR,RAM,ROM,PRIMARY CAMERA,SECONDARY CAMERA,DISPALY SIZE,BATTERY,Price,url
1,SAMSUNG,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB) (4...",Waterfall Blue,4 GB,64 GB,50MP + 5MP + 2MP,Yes,16.76 cm (6.6 inch),6000 mAh,"₹11,999",https://www.flipkart.com/samsung-galaxy-f13-wa...
2,APPLE,"APPLE iPhone 13 (Starlight, 128 GB)",Starlight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹69,900",https://www.flipkart.com/apple-iphone-13-starl...
3,SAMSUNG,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB) (4...",Nightsky Green,4 GB,64 GB,50MP + 5MP + 2MP,Yes,16.76 cm (6.6 inch),6000 mAh,"₹11,999",https://www.flipkart.com/samsung-galaxy-f13-ni...
4,SAMSUNG,"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB) (4...",Sunrise Copper,4 GB,64 GB,50MP + 5MP + 2MP,Yes,16.76 cm (6.6 inch),6000 mAh,"₹11,999",https://www.flipkart.com/samsung-galaxy-f13-su...
5,APPLE,"APPLE iPhone 11 (White, 64 GB)",-,A13 Bionic Chip,iOS 14.2,-,-,"iPhone, USB-C to Lightning Cable, Documentation","AAC LC, HE AAC, HE AAC v2, Protected AAC, MP3,...","₹43,900",https://www.flipkart.com/apple-iphone-11-white...
6,APPLE,"APPLE iPhone 13 (Green, 128 GB)",Green,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹69,900",https://www.flipkart.com/apple-iphone-13-green...
7,APPLE,"APPLE iPhone 13 (Midnight, 128 GB)",Midnight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹69,900",https://www.flipkart.com/apple-iphone-13-midni...
8,SAMSUNG,"SAMSUNG Galaxy S21 FE 5G (Olive, 128 GB) (8 G...",Olive,8 GB,128 GB,12MP + 12MP + 8MP (OIS),Yes,16.26 cm (6.4 inch),4500 mAh,"₹54,999",https://www.flipkart.com/samsung-galaxy-s21-fe...
9,POCO,"POCO C31 (Royal Blue, 64 GB) (4 GB RAM)",-,-,-,-,-,-,-,"₹9,499",https://www.flipkart.com/poco-c31-royal-blue-6...
10,POCO,"POCO C31 (Royal Blue, 32 GB) (3 GB RAM)",-,-,-,-,-,-,-,"₹8,499",https://www.flipkart.com/poco-c31-royal-blue-3...


In [140]:
# saving dataframe as csv file
df.to_csv('smartphone.csv')

In [141]:
driver.close()

Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [114]:
city = input('enter city : ')

enter city : meerut


In [115]:
driver = webdriver.Chrome(r'chromedriver.exe')  # connecting to web driver
time.sleep(1)
driver.get('https://www.google.com/maps')
time.sleep(1)
search = driver.find_element(By.XPATH,'//input[@class="tactile-searchbox-input"]')
search.send_keys(city)
time.sleep(1)
search_clk = driver.find_element(By.XPATH,'//button[@class="mL3xi"]')
search_clk.click()

In [121]:
try:
    c_url = driver.current_url
    print(" current url" ,c_url)
    lat_lng = re.findall(r'@(.*data)',c_url)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitute = {}, Longitude = {}".format(lat,lng)) 

except NoSuchElementException:
    print("error", str(e))

 current url https://www.google.com/maps/place/Meerut,+Uttar+Pradesh/@28.9872441,77.6289198,12z/data=!4m5!3m4!1s0x390c64f457b66325:0x42faa83387a6be5e!8m2!3d28.9844618!4d77.7064137
Latitute = 28.9872441, Longitude = 77.6289198


In [122]:
driver.close()

Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [46]:
# connecting to webdriver
driver = webdriver.Chrome(r'chromedriver.exe') 

In [47]:
# going to the site
driver.get('https://trak.in./')

In [50]:
# click on funding deals
funding = driver.find_element(By.XPATH,'//a[@title="https://trak.in/india-startup-funding-investment-2015/"]')
funding.click()

In [57]:
# serial no
s_no = []
rank =  driver.find_elements(By.XPATH,'//td[@class="column-1"]')
for i in rank:
    s_no.append(i.text)
s_no=s_no[5:29]    

In [59]:
# date
date = []
d = driver.find_elements(By.XPATH,'//td[@class="column-2"]')
for i in d:
    date.append(i.text)
date = date[5:29]        

In [61]:
# startup name
s_name = []
s = driver.find_elements(By.XPATH,'//td[@class="column-3"]')
for i in s:
    s_name.append(i.text)
s_name = s_name[5:29] 

In [70]:
# industary/vertical
industary = []
ind = driver.find_elements(By.XPATH,'//td[@class="column-4"]')
for i in ind:
    industary.append(i.text)
industary = industary[5:29] 

In [72]:
# sub-vertical
sub = []
su = driver.find_elements(By.XPATH,'//td[@class="column-5"]')
for i in su:
    sub.append(i.text)
sub = sub[5:29]

In [77]:
# city/location
city = []
ci = driver.find_elements(By.XPATH,'//td[@class="column-6"]')
for i in ci:
    city.append(i.text)
city = city[5:29]

In [79]:
# investor's name
invertor = []
inv = driver.find_elements(By.XPATH,'//td[@class="column-7"]')
for i in inv:
    invertor.append(i.text)
invertor = invertor[5:29]

In [81]:
# investment type
investment = []
invs = driver.find_elements(By.XPATH,'//td[@class="column-8"]')
for i in invs:
    investment.append(i.text)
investment = investment[5:29]

In [83]:
# amount (in USD)
amount = []
am = driver.find_elements(By.XPATH,'//td[@class="column-9"]')
for i in am:
    amount.append(i.text)
amount = amount[5:29]

In [88]:
# creating dataframe
df = pd.DataFrame({'serial no':s_no,'date':date,'startup name':s_name,'industary/vertical':industary,'sub-vertical':sub,'city/location':city,'investor name':invertor,'investment type':investment,'amount (in USD)':amount},index=list(range(1,len(city)+1)))
print('Indian Startup Funding & Investment Chart [2021] from JAN to MARCH in descending order')
df

Indian Startup Funding & Investment Chart [2021] from JAN to MARCH in descending order


,serial no,date,startup name,industary/vertical,sub-vertical,city/location,investor name,investment type,amount (in USD)
1,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
2,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
3,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
4,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
5,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
6,6,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
7,7,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
8,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
10,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"


In [89]:
df.to_csv('startup_funding.csv')

In [90]:
driver.close()

Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
driver = webdriver.Chrome(r'chromedriver.exe')                            # connecting to webdriver   
time.sleep(1)
driver.get('https://www.digit.in/')                                       # connecting to site
time.sleep(2)

In [4]:
gaming_lap = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div[2]/div[7]/ul/li[9]/a')  # click on gaming laptop
gaming_lap.click() 

In [53]:
# urls of laptops
url = []
urls = driver.find_elements(By.XPATH,'//div[@class="left_side"]/a')
for i in urls:
    url.append(i.get_attribute('href'))

In [54]:
url

['https://www.digit.in/laptops/msi-titan-gt77-12uhs-12th-gen-core-i9-12900hx-2022-price-303891.html',
 'https://www.digit.in/laptops/dell-new-alienware-x17-r2-d569944win9-12th-gen-core-i9-12900hk-2022-price-309253.html',
 'https://www.digit.in/laptops/acer-predator-triton-500-se-12th-gen-core-i7-12700h-2022-price-305711.html',
 'https://www.digit.in/laptops/omen-by-hp-16-b1371tx-12th-gen-core-i7-12700h-2022-price-290829.html',
 'https://www.digit.in/laptops/acer-nitro-5-an515-45-nh-qbrsi-007-ryzen-9-5900hx-2022-price-305725.html',
 'https://www.digit.in/laptops/msi-delta-15-5th-gen-ryzen-9-5900hx-2022-price-288855.html',
 'https://www.digit.in/laptops/hp-omen-16-c0141ax-ryzen-9-5900hx-2022-price-289331.html',
 'https://www.digit.in/laptops/lenovo-legion-5-pro-82rf00mgin-12th-gen-core-i7-12700h-2022-price-311997.html',
 'https://www.digit.in/laptops/dell-alienware-m15-r5-gaming-ryzen-r7-5800h-2022-price-288001.html',
 'https://www.digit.in/laptops/lenovo-legion-s7-ryzen-7-5800h-2022-pri

In [107]:
processor = []
laptop_name = []
memory = []
display = []
os = []
price = []

In [108]:
for u in url:
    driver.get(u)
    time.sleep(1)
    # processor
    try:
        pro = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[8]/div/div[1]/div[6]/table/tbody/tr[1]/td[3]')
        processor.append(pro.text)
    except NoSuchElementException:
        pro1 = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[7]/div/div[1]/div[6]/table/tbody/tr[1]/td[3]')
        processor.append(pro1.text)
    # laptop name
    try:
        name = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[2]/h1')
        laptop_name.append(name.text)
    except NoSuchElementException:
        laptop_name.append('-')
    
    # memory :
    try:
        mem = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[3]/div[2]/div[3]/div/ul/li[4]/div/p[2]/strong')
        memory.append(mem.text)
    except NoSuchElementException:
        memory.append('-')
     # display
    try:
        dis = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[3]/div[2]/div[3]/div/ul/li[2]/div/p[2]/strong')
        display.append(dis.text)
    except NoSuchElementException:
        display.append('-')
    # operating system
    try:
        osys = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[3]/div[2]/div[3]/div/ul/li[1]/div/p[2]/strong')
        os.append(osys.text)
    except NoSuchElementException:
        os.append('-')
    # price    
    try:
        p = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[3]/div[2]/div[4]/div/h2/strong')
        price.append(p.text)
    except NoSuchElementException:
        price.append('-')    

In [116]:
df = pd.DataFrame({"LAPTOP NAME":laptop_name,"PROCESSOR":processor,"OPERATING SYSTEM":os,"MEMORY":memory,"DISPLAY":display,"PRICE":price},index=list(range(1,len(price)+1)))
df

,LAPTOP NAME,PROCESSOR,OPERATING SYSTEM,MEMORY,DISPLAY,PRICE
1,MSI Titan GT77 12UHS 12th Gen Core i9-12900HX ...,12th Gen Intel Core i9-12900HX,Windows 11 Home,2 TB SSD/64 GB DDR5,"17.3"" (3840 X 2160)",-
2,Dell New Alienware x17 R2 D569944WIN9 12th Gen...,12th Gen Intel Core i9-12900H,Windows 11 Home,1 TB SSD/32 GB DDR5,"17.3"" (1920 X 1080)","₹ 389,990"
3,Acer Predator Triton 500 SE 12th Gen Core i7-1...,12th Gen Intel Core i7-12700H,Windows 11 Home,2 TB SSD/32 GB DDR5,"16"" (2560 X 1600)","₹ 300,000"
4,HP OMEN 16-B1371TX 12th Gen core i7-12700H (2022),12th Gen Intel Core i7-12700H,Windows 11 Home,1 TB SSD/8 GB DDR5,"16.1"" (2560 X 1440)","₹ 171,490"
5,Acer Nitro 5 AN515-45 NH.QBRSI.007 Ryzen 9-590...,AMD Ryzen 9-5900HX,Windows 11 Home,512 GB SSD/16 GB DDR4,"15.6"" (2560 X 1440)","₹ 172,999"
6,MSI Delta 15 5th Gen Ryzen 9-5900HX (2022),AMD 5th Gen Ryzen 9-5900HX,Windows 11 Home,1 TB SSD/16 GBGB DDR4,"15.6"" (1920 X 1080)","₹ 188,990"
7,HP Omen 16-C0141AX Ryzen 9 5900HX (2022),AMD Ryzen™ 9 5900HX,Windows 11 Home,1 TB NVMe/16 GB DDR4,"16.1"" (2560 X 1440)","₹ 124,990"
8,Lenovo Legion 5 Pro 82RF00MGIN 12th Gen Core i...,12th Gen Intel Core i7-12700H,Windows 11 Home,1 TB SSD/16 GB DDR5,"16"" (2560 X 1600)","₹ 169,990"
9,DELL ALIENWARE M15 R5 GAMING Ryzen R7-5800H (2...,AMD Ryzen R7-5800H,Windows 11 Home,512 GB SSD/16 GB DDR4,"15.6"" (1920 X 1080)","₹ 144,990"
10,Lenovo Legion S7 82K800E8IN Ryzen 7-5800H (2022),Lenovo Ryzen 7-5800H,Windows 11 Home,1 TB SSD/16 GB DDR4,15.6 MP | NA,"₹ 131,990"


In [117]:
df.to_csv('laptop.scv') # saving the data frame

In [118]:
driver.close()

Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [130]:
driver = webdriver.Chrome(r'chromedriver.exe') # connecting to driver

In [131]:
driver.get('https://www.forbes.com/billionaires/') # geting site

In [134]:
# rank
rank = []
r = driver.find_elements(By.XPATH,'//div[@class="rank"]')
for i in r:
    rank.append(i.text)

In [139]:
len(rank)

200

In [136]:
# name
name = []
n = driver.find_elements(By.XPATH,'//div[@class="personName"]')
for i in n:
    name.append(i.text)

In [138]:
len(name)

200

In [140]:
# net worth
worth = []
w = driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
for i in w:
    worth.append(i.text)

In [142]:
len(worth)

200

In [146]:
# country
country = []
c = driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
for i in c:
    country.append(i.text)

In [148]:
len(country)

200

In [149]:
# age 
age = []
a = driver.find_elements(By.XPATH,'//div[@class="age"]')
for i in a:
    age.append(i.text)

In [151]:
len(age)

200

In [152]:
# source
source = []
s = driver.find_elements(By.XPATH,'//span[@class="source-text"]')
for i in s :
    source.append(i.text)

In [154]:
len(source)

200

In [155]:
# industary
industaries = []
i = driver.find_elements(By.XPATH,'//div[@class="category"]')
for k in i:
    industaries.append(k.text)   

In [157]:
len(industaries)

200

In [161]:
df = pd.DataFrame({"Rank":rank, "Name":name, "Net worth":worth, "Age":age, "Country":country, "Source":source, "Industry":industaries})
print('The Richest People In The World ')
df

The Richest People In The World 


,Rank,Name,Net worth,Age,Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [166]:
df

,Rank,Name,Net worth,Age,Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [167]:
df.to_csv('richest people.csv')

In [168]:
driver.close()

Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [3]:
driver = webdriver.Chrome(r'chromedriver.exe') # connecting to driver

In [4]:
driver.get('https://www.youtube.com/watch?v=X3paOmcrTjQ') # opening youtube video

In [12]:
comment = []
time = []
upvote = []

In [52]:
# extracting comments
for  _ in range(50):
    driver.execute_script("window.scrollBy(0,50)")
    try:
        Comment = driver.find_elements(By.XPATH,'//div[@class="style-scope ytd-expander"]')
        for i in Comment[0:500]:
            comment.append(i.text)
    except NoSuchElementException:
        comment.append('-')
    # time 
    try: 
        Time = driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
        for j in Time[0:500]:
            time.append(j.text)
    except NoSuchElementException:
        time.append('-')
    # upvote
    try:
        Upvote = driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
        for k in Upvote[0:500]:
            upvote.append(k.text)
    except NoSuchElementException:
        upvote.append('-')

In [58]:
comment=comment[0:500]

In [61]:
upvote=upvote[0:500]

In [62]:
time=time[0:500]

In [63]:
df = pd.DataFrame({"comment":comment,"Time":time,"Upvote":upvote},index=list(range(1,len(time)+1)))
print('comments on youtube video')
df

comments on youtube video


,comment,Time,Upvote
1,,,43
2,,,600
3,Explore Our FREE Courses With Completion Certi...,,4
4,Great video! As an established data scientist ...,,38
5,This is really a very clear presentation of wh...,,72
...,...,...,...
496,Great video! As an established data scientist ...,1 year ago,28
497,This is really a very clear presentation of wh...,1 year ago,7
498,Great video. You did great explaining this. Th...,1 year ago,3
499,it was a great overview of all the process. i...,1 year ago,


In [64]:
df.to_csv("comments.csv")

In [66]:
driver.close()

Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [3]:
driver = webdriver.Chrome(r'chromedriver.exe') # connecting to driver

In [4]:
driver.get('https://www.hostelworld.com/') # open site

In [5]:
search_hotel = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input')
search_hotel.send_keys("London")

In [6]:
# click on london
london_click = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')
london_click.click()

In [7]:
# click on let's go
go = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[5]/button')
go.click()

In [10]:
# url of hotels
start = 0
end = 1
url = []
for page in range(start,end):
    urls = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for i in urls:
        url.append(i.get_attribute('href'))
    next_button = driver.find_element(By.XPATH,'//i[@class="core-icon icon-core-chevron-right"]')
    next_button.click() 
    time.sleep(3)

In [29]:
# distance from city
start = 0
end = 1
for page in range(start,end):
    dist = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for i in dist:
        distance.append(i.text)
    next_button = driver.find_element(By.XPATH,'//div[@class="pagination pagination"]')
    next_button.click() 
    time.sleep(3)

In [36]:
private_price.clear()

In [12]:
hostel_name = []
distance = []
ratings = []
toatal_review = []
private_price = []
dorms_price = []
facilities = []
properties_description = []

In [47]:
for u in url:
    driver.get(u)
    time.sleep(2)

    # hostel name
    try:
        name = driver.find_element(By.XPATH,'//div[@class="title-2"]')
        hostel_name.append(name.text)
    except NoSuchElementException:
        hostel_name.append('-')
    
    # rating
    try:
        rate = driver.find_element(By.XPATH,'//div[@class="score orange big"]')
        ratings.append(rate.text)
    except NoSuchElementException:
        ratings.append('-')
    
    #total reviews
    try:
        rev = driver.find_element(By.XPATH,'//div[@class="reviews"]')
        toatal_review.append(rev.text)
    except NoSuchElementException:
        toatal_review.append('-')
       
    
    # private_price
    try:
        pp = driver.find_element(By.XPATH,'//div[@class="price"]')
        private_price.append(pp.text)
    except NoSuchElementException:
        private_price.append('-')   
     
    # dorms_price
    try:
        dp = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/section/div[8]/div/div/section/div/div[1]/div[3]/div[2]/div/div[2]/div[1]/div/div[1]/div[1]')
        dorms_price.append(dp.text)
    except NoSuchElementException:
        dorms_price.append('-')    
     
    #facilities
    try:
        f = driver.find_element(By.XPATH,'//ul[@class="facilities"]')
        facilities.append(f.text)
    except NoSuchElementException:
        facilities.append('-')    
    

    # properties_description
    try:
        pd = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/section/div[6]/div/div[2]/div/div/div[2]')
        properties_description.append(pd.text.replace('\n',' '))
    except NoSuchElementException:
        properties_description.append('-')    

In [93]:
# creating data frame
df = pd.DataFrame({'hostel name':hostel_name,
                   'distance from city':distance,
                   'toatal review':toatal_review,
                   'ratings':ratings,
                   'private price':private_price,
                   'dorms price':dorms_price,
                   'facilities':facilities,
                   'properties_description':properties_description},index = list(range(1,len(properties_description)+1)))
print('hostels in london : ')
df

hostels in london : 


,hostel name,distance from city,toatal review,ratings,private price,dorms price,facilities,properties_description
1,Wombat's City Hostel London,Hostel - 3.6km from city centre,14116 Total Reviews,9.0,Rs3387.29,-20%,Linen Included Free City Maps Free WiFi Free I...,A safe haven in the middle of the metropolis: ...
2,St Christopher's Village,Hostel - 1.8km from city centre,11536 Total Reviews,8.2,Rs2058.84,-,Linen Included Free City Maps Free WiFi Free I...,COVID 19 Policy Update. In response to Coronav...
3,Urbany Hostel London,Hostel - 5.4km from city centre,476 Total Reviews,9.3,Rs18237.54,Rs18237.54,Linen Included Free WiFi,"Welcome to Urbany Hostel London, our first int..."
4,Generator London,Hostel - 3km from city centre,7167 Total Reviews,7.8,Rs22169.97,-,Linen Included Free City Maps Free WiFi Free I...,Generator London is a design hotel-hostel loca...
5,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,4523 Total Reviews,7.0,Rs2705.42,-,Linen Included Free City Maps Free WiFi,Safestay at Elephant & Castle is ideal if you ...
6,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,1325 Total Reviews,6.5,Rs2352.58,-,Linen Included Free WiFi,Safestay Holland Park Safestay Holland Park i...
7,Phoenix Hostel,Hostel - 4.2km from city centre,3769 Total Reviews,7.3,Rs2143.52,-10%,Linen Included Free City Maps Free WiFi Free I...,"The Phoenix Bar & Hostel, housed in a homely E..."
8,St Christopher's Camden,Hostel - 4.3km from city centre,3842 Total Reviews,8.2,Rs2865.96,-10%,Linen Included Free City Maps Free WiFi Free I...,COVID 19 Policy Update. In response to Coronav...
9,St Christopher's Hammersmith,Hostel - 7.5km from city centre,4118 Total Reviews,7.8,Rs1960.92,-10%,Linen Included Free City Maps Free WiFi Free I...,"In response to Coronavirus (COVID-19), additio..."
10,NX London Hostel,Hostel - 6.1km from city centre,1232 Total Reviews,7.6,Rs7204.16,Rs7204.16,Free Breakfast Linen Included Free Parking Fre...,Welcome to NX London Hostel! A Friendly backp...


In [94]:
df.to_csv('hostel.csv')

In [95]:
driver.close()